<a href="https://colab.research.google.com/github/FinnRobertson15/PatternAnalysis-2023/blob/topic-recognition/Colab%20version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
%ls

drive/  sample_data/


In [9]:
%cd /content/drive/MyDrive/AD_NC

/content/drive/MyDrive/AD_NC


In [ ]:
!unzip NC.zip

unzip:  cannot find or open NC.zip, NC.zip.zip or NC.zip.ZIP.


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import time
import numpy as np
# import pandas as pd
import math
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
import torch
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from PIL import Image
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import ConcatDataset

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_name)

        if self.transform:
            image = self.transform(image)

        return image

class CustomConcatDataset(ConcatDataset):
    def __init__(self, datasets):
        super(CustomConcatDataset, self).__init__(datasets)

    def __getitem__(self, index):
        item1 = super(CustomConcatDataset, self).__getitem__(index)
        item2 = super(CustomConcatDataset, self).__getitem__(index)
        return item1, item2

def combine(AD, NC):
    # Set labels for the AD dataset
    labels_AD = [True] * len(AD)
    # Set labels for the NC dataset
    labels_NC = [False] * len(NC)

    # Combine the datasets and labels
    X = CustomConcatDataset([AD, NC])
    Y = labels_AD + labels_NC
    # X = torch.utils.data.RandomSampler(X)
    seed = 123
    torch.manual_seed(seed)
    np.random.seed(seed)


    X_loader = DataLoader(X, batch_size=128, shuffle=True)
    Y_loader = DataLoader(Y, batch_size=128, shuffle=True)
    return X_loader, Y_loader

size = 256

transform_X = transforms.Compose([
    transforms.Resize((size, size)),  # Resize the image to the desired size
    transforms.ToTensor(),         # Convert the image to a tensor
    transforms.Lambda(lambda x: x / 255.0),
    transforms.Lambda(lambda x: (x - x.mean()) / x.std())  # Subtract mean and divide by standard deviation
])



# Replace 'your_nii_folder' with the path to your folder containing .nii files
# nii_dataset = NiiDataset(root_dir=r'keras_png_slices_data\keras_png_slices_data\keras_png_slices_train')
loaders = {}

for stage in ['test']:
    loaders[stage] = {}
    AD = CustomDataset(root_dir=rf'{stage}/AD', transform=transform_X)
    NC = CustomDataset(root_dir=rf'{stage}/NC', transform=transform_X)
    loaders[stage]['X'], loaders[stage]['Y'] = combine(AD, NC)
# Y_train  = CustomDataset(root_dir=r'ADNI_AD_NC_2D\AD_NC\train\NC', transform=transform_Y)
# X_train_AD_loader = DataLoader(X_train_AD, batch_size=4, shuffle=True)
# Y_train_loader = DataLoader(Y_train, batch_size=4, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the model
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.maxpool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.maxpool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 64 * 64, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = self.maxpool1(x)
        x = nn.functional.relu(self.conv2(x))
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.softmax(self.fc2(x), dim=1)
        return x



# Instantiate the model
model = CustomModel()



In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
total_step = len(loaders['train']['X'])
sched_linear_1 = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=0.005, max_lr=learning_rate, step_size_up=15, step_size_down=15, mode="triangular", verbose=False)
sched_linear_3 = torch.optim.lr_scheduler.LinearLR(optimizer, start_factor=0.005/learning_rate, end_factor=0.005/learning_rate, verbose=False)
scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer, schedulers=[sched_linear_1, sched_linear_3], milestones=[30])


In [18]:
import torch
import torch.nn as nn
import torchvision.models as models

class SiameseNetwork(nn.Module):
    def __init__(self, pretrained=True):
        super(SiameseNetwork, self).__init__()
        self.resnet = models.resnet18(pretrained=pretrained)
        self.linear = nn.Linear(1000, 2)  # 1000 is the number of output features from ResNet

    def forward(self, x1, x2):
        output1 = self.resnet(x1)
        output2 = self.resnet(x2)
        return self.linear(torch.abs(output1 - output2))

model = SiameseNetwork()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 220MB/s]


In [19]:
class ContrastiveLoss(torch.nn.Module):
    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                                      label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss_contrastive


In [26]:
import torch.optim as optim

# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize the network, loss function, and optimizer
siamese_net = SiameseNetwork(pretrained=True).to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(siamese_net.parameters(), lr=0.0005)

# Training loop
epochs = 2
for epoch in range(epochs):
  for i, (images, label) in enumerate(zip(loaders['test']['X'], loaders['test']['Y'])):
    img0, img1 = images
    img0, img1, label = img0.to(device), img1.to(device), label.to(device)
    output1, output2 = model(img0, img1)


    # Adjust the shapes to match the criterion
    # outputs = outputs.view(-1)  # Flatten the outputs to a 1D tensor
    # seg = seg.view(-1)  # Flatten the seg tensor to a 1D tensor
    loss = criterion(output1, output2, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 5 == 0:
      print(f"Epoch [{epoch + 1} / {num_epochs}], Step [{i + 1} / {total_step} Loss {loss.item()}]")

RuntimeError: ignored

In [17]:
num_epochs = 2
for epoch in range(num_epochs):

  for i, (images, label) in enumerate(zip(loaders['test']['X'], loaders['test']['Y'])):
    img0, img1 = images
    img0, img1, label = img0.to(device), img1.to(device), label.to(device)
    outputs = model(images)


    # Adjust the shapes to match the criterion
    # outputs = outputs.view(-1)  # Flatten the outputs to a 1D tensor
    # seg = seg.view(-1)  # Flatten the seg tensor to a 1D tensor
    loss = criterion(outputs, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 5 == 0:
      print(f"Epoch [{epoch + 1} / {num_epochs}], Step [{i + 1} / {total_step} Loss {loss.item()}]")

  scheduler.step()
end = time.time()

NameError: ignored